<a href="https://colab.research.google.com/github/Pt-home/Notebooks/blob/main/Rashid_7_WithRotationEpohsAndSave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy
import scipy.special
import scipy.ndimage
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class neuralNetwork:

  def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
    self.inodes = inputnodes
    self.hnodes = hiddennodes
    self.onodes = outputnodes

    self.lr = learningrate

    self.wih = (numpy.random.rand(self.hnodes, self.inodes) - 0.5)
    self.who = (numpy.random.rand(self.onodes, self.hnodes) - 0.5)

    self.activation_function = lambda x: scipy.special.expit(x)

  def train(self, inputs_list, targets_list):
    # inputs -> matrix
    inputs = numpy.array(inputs_list, ndmin=2).T
    targets = numpy.array(targets_list, ndmin=2).T

    #input for hidden
    hidden_inputs = numpy.dot(self.wih, inputs)
    #output at hidden
    hidden_outputs = self.activation_function(hidden_inputs)

    #input for final
    final_inputs = numpy.dot(self.who, hidden_outputs)
    #output at final
    final_outputs = self.activation_function(final_inputs)

    #output_errors
    output_errors = targets - final_outputs

    #hidden_errors
    hidden_errors = numpy.dot(self.who.T, output_errors)

    #refresh who
    self.who += self.lr * numpy.dot(output_errors*final_outputs*(1-final_outputs), numpy.transpose(hidden_outputs))

    #refresh wih
    self.wih += self.lr * numpy.dot(hidden_errors*hidden_outputs*(1-hidden_outputs), numpy.transpose(inputs))

  def query(self, inputs_list):
    # input -> matrix
    inputs = numpy.array(inputs_list, ndmin=2).T

    #input for hidden
    hidden_inputs = numpy.dot(self.wih, inputs)
    #output at hidden
    hidden_outputs = self.activation_function(hidden_inputs)

    #input for final
    final_inputs = numpy.dot(self.who, hidden_outputs)
    #output at final
    final_outputs = self.activation_function(final_inputs)

    return  final_outputs




In [6]:
inputnodes = 28 * 28
hiddennodes = 300
outputnodes = 10

learningrate = 0.1

epochs = 5

n = neuralNetwork(inputnodes, hiddennodes, outputnodes, learningrate)

In [ ]:
#Training

In [7]:
training_data_file = open("mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

In [9]:
for e in range(epochs):
  print(f"Epochs: {e+1}")
  for record in training_data_list:

    all_values = record.split(",")
    inputs = (numpy.asarray(all_values[1:], dtype=float)/255.0 * 0.99) + 0.01
    targets = numpy.zeros(outputnodes) + 0.01
    targets[int(all_values[0])] = 0.99
    n.train(inputs, targets)

    plus_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),
                                                10, cval=0.01,
                                                order=1, reshape=False)
    n.train(plus_img.reshape(28*28), targets)
    minus_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),
                                                -10, cval=0.01,
                                                order=1, reshape=False)
    n.train(minus_img.reshape(28*28), targets)

Epochs: 1


<ipython-input-9-85324e4afb21>:11: DeprecationWarning: Please import `rotate` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  plus_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),
<ipython-input-9-85324e4afb21>:15: DeprecationWarning: Please import `rotate` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  minus_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),


Epochs: 2
Epochs: 3
Epochs: 4
Epochs: 5


In [ ]:
#Testing

In [10]:
test_data_file = open("mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

In [11]:
scoreboard = []

for record in test_data_list:

  all_values = record.split(",")

  correct_label = int(all_values[0])

  inputs = (numpy.asarray(all_values[1:], dtype=float)/255.0 * 0.99) + 0.01

  outputs = n.query(inputs)

  label = numpy.argmax(outputs)
  if label == correct_label:
     scoreboard.append(1)
  else:
     scoreboard.append(0)

scoreboard_array = numpy.asarray(scoreboard)
print("Эффективность: ", scoreboard_array.sum() / scoreboard_array.size )

Эффективность:  0.9708


In [ ]:
n.wih.shape

(300, 784)

In [ ]:
wih  = n.wih.flatten()
wih = list(wih)
len(wih)

235200

In [ ]:
n.who.shape

(10, 300)

In [ ]:
who  = n.who.flatten()
who = list(who)
len(who)

3000

In [ ]:
f = open(f"wih_rot_{hiddennodes}_{learningrate}_{epochs}.dat", "w")
for i in range(len(wih)-1):
  f.write(f"{wih[i]}, ")
f.write(f"{wih[-1]}")
f.close()

In [ ]:
f = open(f"who_rot_{hiddennodes}_{learningrate}_{epochs}.dat", "w")
for i in range(len(who)-1):
  f.write(f"{who[i]}, ")
f.write(f"{who[-1]}")
f.close()